## Welcome to NERDSS on Google Colab!
*Author: Yue Moon Ying*

Upon running, Google Colab will ask you for access to Google Drive. Once mounted, you can access your Google Drive files under the /content/drive/My Drive directory.

Note, currently using Public version of NERDSS
Be careful that ioNERDSS and NERDSS will be installed and complied in YOUR google drive which takes around 200 MB of space
Remember to reduce the iteration number to accomadate computational power on Google colab
Be aware there is some latency between uploading files to Google Drive and sync to Mount

### 1. Mount Google Drive and Download NERDSS

In [ ]:
# Mount Google Drive
# Accessing Google Drive from a Google Colab notebook

from google.colab import drive
drive.mount('/content/drive/')

# Navigate to Your Drive Files
# Google Colab will ask you for access to Google Drive
# Once mounted, you can access your Google Drive files under the /content/drive/My Drive directory.

# Change to the desired directory
import os
os.chdir('/content/drive/My Drive')

# Pull NERDSS from Github
!git clone https://github.com/mjohn218/NERDSS.git
!git fetch https://github.com/mjohn218/NERDSS.git

# List files in NERDSS folder
os.chdir('/content/drive/My Drive/NERDSS')
!ls

# Verify current working directory
print(os.getcwd())

In [ ]:
# Check C++ compiler
!g++ --version

# Install Dependency: GSL
!sudo apt install libgsl-dev -y

### 2. Compile NERDSS

In [ ]:
# Compile
# Under free Google Colab this takes est. 12 minutes
!make serial -j8

### 3. Download the ionerdss and install required packages

In [ ]:
os.chdir('/content/drive/My Drive')
!git clone https://github.com/JohnsonBiophysicsLab/ionerdss.git
!git fetch https://github.com/JohnsonBiophysicsLab/ionerdss.git
os.chdir('/content/drive/My Drive/ionerdss')
!ls
# install all dependencies in requirements.txt
%pip install -r requirements.txt

In [ ]:
import sys
sys.path.insert(0, os.path.abspath('/content/drive/My Drive/ionerdss'))

from ionerdss.model_setup.protein_model import ProteinModel
print('ionerdss.model_setup.protein_model imported successfully!')

### 4. Prepare the NERDSS input files
we will use pdb id 8y7s as an example (it is a homo 4-mer; each monomer will be represented by the same NERDSS molecule template)

In [ ]:
pdb_id = './data/7uhy.cif'
model = ProteinModel(pdb_id)
print('ProteinModel created!')

In [ ]:
import nglview as nv

view = nv.show_file(pdb_id)
view

We can now detect binding interfaces in the structure.
- `cutoff=0.35` is the maximum distance (in nm) for atom contacts.
- `residue_cutoff=3` means we need at least 3 contacting residues.

In [ ]:
model.verbose = False  # set to True for more outputs information
model.detect_interfaces(cutoff=0.35, residue_cutoff=3)
print('Interfaces detected!')

show each chain’s **Center of Mass** and **Interface** points

In [ ]:
%matplotlib widget

model.plot_original_coarse_grained_structure()

If the structure has homologous chains, we will regularize them so they share the same geometry.
This step also sets up Reaction objects if two chains can bind.

In [ ]:
model.regularize_molecules(dist_thresh_intra=3.5, dist_thresh_inter=3.5, angle_thresh=25)

print('After regularization:')
model.plot_regularized_structure()

Generates the NERDSS Input files

In [ ]:
model.generate_nerdss_ready_files()

### 5. Run NERDSS simulation

In [ ]:
!cp /content/drive/My Drive/NERDSS/bin/nerdss /content/drive/My Drive/ionerdss
!chmod +x .nerdss
!./nerdss -f ./parms.inp